In [112]:
import numpy as np

def diag_2d(Y_prim_list):

    N_cols = 0

    for Y_prim in Y_prim_list:
        N_cols += Y_prim.shape[1]

    Y_prims = np.zeros((N_cols,N_cols))+0j

    it = 0
    for Y_prim in Y_prim_list:
        N = Y_prim.shape[0] 
        Y_prims[it:(it+N),it:(it+N)] = Y_prim
        it += N


    return Y_prims

In [85]:
z_a = 0.01*(1+6j)*1.0**2/1.5e6
z_b = 0.01*(1+6j)*1.0**2/1.5e6
z_c = 0.01*(1+6j)*1.0**2/1.5e6
U_1 = 7200.0
U_2 = 4160.0

In [86]:
Z_B = np.array([
[z_a, 0.0, 0.0],
[0.0, z_b, 0.0],
[0.0, 0.0, z_c],]) 

In [108]:
B = np.array([
[ 1, 0, 0],
[-1, 0, 0],
[ 0, 1, 0],
[ 0,-1, 0],
[ 0, 0, 1],
[ 0, 0,-1]])


N_a = np.array([
[ 1/U_1,     0],
[-1/U_1,     0],
[     0, 1/U_2],
[     0,-1/U_2],
])

N_row_a = np.hstack((N_a,np.zeros((4,4))))
N_row_b = np.hstack((np.zeros((4,2)),N_a,np.zeros((4,2))))
N_row_c = np.hstack((np.zeros((4,4)),N_a))

N = np.vstack((N_row_a,N_row_b,N_row_c))

In [109]:
Y_1 = B @ np.linalg.inv(Z_B) @ B.T
Y_w = N @ Y_1 @ N.T

In [157]:
A_trafo = np.zeros((7,12))
A_trafo[0,0] = 1.0
A_trafo[1,4] = 1.0
A_trafo[2,8] = 1.0

A_trafo[3,1] = 1.0
A_trafo[3,5] = 1.0
A_trafo[3,9] = 1.0

A_trafo[4,2] = 1.0
A_trafo[4,11] = 1.0
A_trafo[5,3] = 1.0
A_trafo[5,6] = 1.0
A_trafo[6,7] = 1.0
A_trafo[6,10] = 1.0

In [201]:
Y_trafo_prim = A_trafo @ Y_w @ A_trafo.T
Y_load_prim = np.eye(3)*1e-6
#Y_load_prim[0,0] = 10.0
#Y_load_prim[1,1] = 10.0

In [193]:
Y_prim = diag_2d([Y_trafo_prim,Y_load_prim])


### Incidence Matrix

In [194]:
id_nodes = np.array(['a_1','b_1','c_1','n_1','a_2','b_2','c_2'])
id_i = np.array(['a_2','b_2','c_2'])
id_v = np.array(['a_1','b_1','c_1','n_1'])

In [195]:
A = np.array([
  [ 1,   0,   0,   0,   0,   0,   0,   0,   0,   0], # a_1  0
  [ 0,   1,   0,   0,   0,   0,   0,   0,   0,   0], # b_1  1
  [ 0,   0,   1,   0,   0,   0,   0,   0,   0,   0], # c_1  2
  [ 0,   0,   0,   1,   0,   0,   0,   0,   0,   0], # n_1  3
  [ 0,   0,   0,   0,   1,   0,   0,   1,   0,   0], # a_2  4
  [ 0,   0,   0,   0,   0,   1,   0,   0,   1,   0], # b_2  5
  [ 0,   0,   0,   0,   0,   0,   1,   0,   0,   1], # c_2  6
])
# trafo                                load

A_v = A[0:4,:]
A_i = A[4:7,:]

Y_ii = A_i @ Y_prim @ A_i.T
Y_iv = A_i @ Y_prim @ A_v.T
Y_vv = A_v @ Y_prim @ A_v.T
Y_vi = A_v @ Y_prim @ A_i.T

inv_Y_ii =  np.linalg.inv(Y_ii)

In [196]:
V_known = np.zeros((4,1))+0j
V_known[id_v=='a_1'] = U_1*np.exp(0.0j)
V_known[id_v=='b_1'] = U_1*np.exp(2.0/3*np.pi*1j)
V_known[id_v=='c_1'] = U_1*np.exp(4.0/3*np.pi*1j)
V_known[id_v=='n_1'] = 0.0

I_known = np.zeros((3,1))+0j
I = 0.0
phi =np.deg2rad(0.0)
I_known[id_i=='a_2'] = I*np.exp((0.0-phi)*1j)
I_known[id_i=='b_2'] = I*np.exp((2.0/3.0*np.pi-phi)*1j)
I_known[id_i=='c_2'] = I*np.exp((4.0/3.0*np.pi-phi)*1j)

#I_known =0*np.array([1.0*np.exp(0.0j),
#                    1.0*np.exp(2.0/3*np.pi*1j),
#                    1.0*np.exp(4.0/3*np.pi*1j)]).reshape((9,1))

In [197]:
V_unknown = inv_Y_ii @ ( I_known - Y_iv @ V_known)
I_unknown =Y_vv @ V_known + Y_vi @ V_unknown

In [198]:
V_known

array([[ 7200.   +0.j        ],
       [-3600.+6235.38290725j],
       [-3600.-6235.38290725j],
       [    0.   +0.j        ]])

In [199]:
for item in ['a_2','b_2','c_2']:
    V = V_unknown[id_i==item,0] 
    print(item,': V = {:2.3f}|{:2.1f}º V'.format(float(np.abs(V)), float(np.angle(V,deg=True))))

a_2 : V = 772.962|-59.0º V
b_2 : V = 772.963|121.0º V
c_2 : V = 3602.665|-90.0º V


In [200]:
for item in ['a_1','b_1','c_1','n_1']:
    I = I_unknown[id_v==item,0]
    print(item,': I = {:2.3f}|{:2.1f}'.format(float(np.abs(I)), float(np.angle(I,deg=True))))

a_1 : I = 2977.337|-59.0
b_1 : I = 1488.669|121.0
c_1 : I = 1488.668|121.0
n_1 : I = 0.000|-135.0
